# Model selection

For the project we want to fine-tune LLM for better reliability. We want to test several approches and choose the best one.

For testing all models we will use the part of Filipp Kirkorov's song "Snow" ("Снег").

In [8]:
song_part = """天上太阳红呀红彤彤诶
心中的太阳是毛泽东诶
他领导我们得解放诶
人民翻身当家做主人
咿呀咿吱呦喂
呀而呀吱呦啊
人民翻身当家做主人"""

Library "langid" will be used to identify language of the song

In [4]:
# !pip install langid
# !pip install sentence_transformers

In [5]:
import langid

langid.classify(song_part)

## Gemini AI

In [6]:
import google.generativeai as genai
from sentence_transformers import SentenceTransformer


In [9]:
API_KEY_GENAI=""

genai.configure(api_key=API_KEY_GENAI)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


In [10]:
def detect_lang(song_text):
  return langid.classify(song_text)[0]

In [11]:
def translate_song(song_text, init_lang, target_lang):
  model = genai.GenerativeModel("gemini-1.5-flash")
  ask  = f"Give me only translation text from {init_lang} to {target_lang} for song text"
  response = model.generate_content(ask + "\n" + song_text)

  return response.text


In [12]:
transl_song = translate_song(song_part, detect_lang(song_part), "en")
transl_song

In [13]:
def summarization(song_text):
  try:
    model = genai.GenerativeModel("gemini-1.5-flash")
    ask  = "Give me only summarization of this song text"
    response = model.generate_content(ask + "\n" + song_text)

    return response.text

  except:

      return 'Sorry, text include forbidden words'


In [14]:
summarization(transl_song)

During our experiment, we unexpetetly noticed that Kirkorov's song contains forbidden words for the GemeniAI 😶

## google/mt5-large

What about simple summarization? We want to see how summarization models could work, too. For this we will use the following model

In [15]:
from transformers import pipeline

def mt5_multilingual_summarization(song_text, language_code="en"):
    try:
        # Load the mT5 model for multilingual summarization
        model_id = "google/mt5-large"

        pipe = pipeline("summarization", model=model_id, tokenizer=model_id, device="cuda")

        # Create a prompt to request summarization
        prompt = f"Summarize the following song text: {song_text}"

        # Generate the summary
        summary = pipe(prompt, max_length=150, min_length=30, do_sample=False)

        # Check if the response is valid
        if summary and 'summary_text' in summary[0]:
            return summary[0]['summary_text'].strip()
        else:
            return "Error: No summary generated or unexpected response structure."

    except Exception as e:
        return f"Error: {str(e)}"


song_text = """Don't kill me
Just help me run away from everyone
I need a place to stay
Where I can cover up my face
Don't cry

I am just a freak!

I am just a freak!

I am just a freak!

I am just a freak!

My head is filled with parasites
Black holes cover up my eyes
I dream of you almost every night
Hopefully, I won't wake up this time

I won't wake up this time!
I won't wake up this time!
I won't wake up this time!"""

summary = mt5_multilingual_summarization(song_text)
print("Summary:", summary)


As we can see, summarization cannot interpret the complex meaning of the song.

## meta-llama/Llama-3.2-1B

It looks like simple promt to make summarization of text not works correctly. Probably, it happens because lyric is not just text, it is deeper, and models cannot understand it at all.

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch.cuda.empty_cache()

model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, token="")
model = AutoModelForCausalLM.from_pretrained(model_name,  token="").to('cuda')

question = "What is the meaning of this song?"

input_text = f"{song_part}\n\nQuestion: {question}\nAnswer:"

inputs = tokenizer(input_text, return_tensors="pt").to('cuda')

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.9,      # Slightly higher for diversity
    top_p=0.9,            # Broader selection range
    num_return_sequences=1
)

answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

answer_only = answer.split("Answer:")[-1].strip()

print(answer_only)

In [19]:
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.9,
    top_p=0.9,
)

answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

answer_only = answer.split("Answer:")[-1].strip()

print(answer_only)

In [21]:
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
)

answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

answer_only = answer.split("Answer:")[-1].strip()

print(answer_only)

According to the outputs, we can see that <b>meta-llama/Llama-3.2-1B</b> shows unpredictible results. In relation to parameters, it could provide different  outputs. In such task, GeminiAI looks more stable.